In [1]:
import torch
from accelerate import Accelerator
from src.datasets_loader import get_dataset, Collator
from src import models

import sys

sys.path.append("../")

from exp_configs import EXP_GROUPS

In [2]:
accelerator = Accelerator()
exp_dict = EXP_GROUPS["base"][0]

In [3]:
train_data = get_dataset(
    "/mnt/colab_public/datasets/joao/CodeSearchNet",
    "train", 
    maximum_raw_length=exp_dict["maximum_raw_length"],
)

No config specified, defaulting to: code_search_net/all
Reusing dataset code_search_net (/mnt/colab_public/datasets/joao/CodeSearchNet/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1880853 [00:00<?, ?ex/s]

In [4]:
train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=exp_dict["train_batch_size"],
        num_workers=exp_dict["n_workers"],
        collate_fn=Collator(
            tokenizer_path=exp_dict["tokenizer_path"],
            maximum_length=exp_dict["maximum_length"],
            mlm_masking_probability=exp_dict["mlm_masking_probability"],
            contrastive_masking_probability=exp_dict["contrastive_masking_probability"],
        ),
        drop_last=True,
    )

In [5]:
exp_dict["vocab_size"] = len(train_loader.collate_fn.tokenizer.vocab)

In [6]:
a = next(iter(train_loader))

In [7]:
model = models.get_model(
        exp_dict=exp_dict,
        accelerator=accelerator,
    )

In [ ]:
model.train_on_loader(train_loader, epoch=0)